**伯努利分布**

\begin{eqnarray*} Pr(X=k|p) = P^k(1-k) \; \mbox{for}\; k\in \{0,1\} \end{eqnarray*}

**二项分布**

\begin{eqnarray*}
Pr(X=k|p) = C_n^kp^k(1-p)^{n-k} \; \mbox{for}\; k\in {0,1,...,n}
\end{eqnarray*}


In [6]:
%pylab
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine 

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
engine = create_engine('postgresql://postgres@localhost:5432/postgres')

In [11]:
m1_sql_query = "select * from  oanda_eur_usd_m1 where dt in \
                (select * from \
                generate_series('2010-07-10 00:00:00', '2010-07-15 23:59:00', interval '1min') \
                                    as b) order by dt;"

In [9]:
df = pd.read_sql_query(m1_sql_query, 
                       con=engine)

In [50]:
instrument = 'EUR_USD'
fromDate = '2010-07-10 00:00:00'
toDate = '2010-07-15 23:59:00'
period = '5min'
fromtbl = 'oanda_%s_m1' % (instrument)

win_agg_group_sql = '''
  with intervals as (
  select start, start + interval '%s' as end
  from generate_series('%s', '%s', interval '%s') as start
  )
  select distinct
  intervals.start as dt,
  min(ask_l) over w as low,
  max(ask_h) over w as high,
  first_value(ask_o) over w as open,
  last_value(ask_c) over w as close,
  sum(v) over w as volume
  from
  intervals
  join %s tmb on
  tmb.dt >= intervals.start and
  tmb.dt < intervals.end
  window w as (partition by intervals.start)
  order by intervals.start;
''' % (period,fromDate,toDate,period,fromtbl)

df = pd.read_sql_query(win_agg_group_sql, 
                       con=engine)
import matplotlib.dates as mdates
pd.to_datetime(df.dt)
df.dt = pd.to_datetime(df.dt)
df = df[['dt', 'open', 'high', 'low', 'close', 'volume']]
df["dt"] = df["dt"].apply(mdates.date2num)
df
#mdates.date2num(df.dt)
#df['dt'].map(mdates.date2num)

,dt,open,high,low,close,volume
0,733964.774306,1.26477,1.26477,1.26447,1.26447,2
1,733964.777778,1.26476,1.26484,1.26451,1.26451,4
2,733964.781250,1.26454,1.26454,1.26451,1.26451,4
3,733964.784722,1.26454,1.26454,1.26451,1.26451,4
4,733964.788194,1.26454,1.26470,1.26451,1.26465,15
5,733964.791667,1.26479,1.26479,1.26280,1.26310,107
6,733964.795139,1.26304,1.26313,1.26285,1.26288,21
7,733964.798611,1.26290,1.26406,1.26290,1.26390,84
8,733964.802083,1.26382,1.26383,1.26333,1.26347,29
9,733964.805556,1.26351,1.26357,1.26330,1.26330,13
